Set covering problem description

In [259]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
import numpy as np

In [260]:
PROBLEM_SIZE = 100
NUM_SETS = 500
SETS = tuple(
    [
        np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)])
        for _ in range(NUM_SETS)
    ]
)
# prob 30% to be true, 70% to be false. This is the problem space.
State = namedtuple("State", ["taken", "not_taken"])

In [261]:
def goal_check(state):
    # this the test if we solve everything. We want each state to be covered. If there is an overlapping it's fine.
    # return np.all(reduce(np.logical_or, [SETS[i] for i in state.taken]))
    return np.all(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )


def distance_prof(state):
    num_not_covered_sets = PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return num_not_covered_sets


def overlap(taken):
    n_overlap = np.sum(np.sum([SETS[i] for i in taken], axis=0) > 1)
    if (
        n_overlap > 1
    ):  # because the first insert has an empty taken set, so we have 0 division.
        normalizer = len(taken) * PROBLEM_SIZE
        n_overlap = n_overlap / normalizer
    return n_overlap


# how far I am from solving the problem
# this is the number of sets that I still need to cover.
def my_h(state):
    num_not_covered_sets = PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    num_of_overlap = overlap(state.taken)

    return len(state.taken) + num_not_covered_sets + num_of_overlap

Allora dobbiamo cercare di ridurre l'overlap. O troviamo un modo per encodarlo nella distance. (potremmo fare in modo di rendere il numero float in questo modo distance.overlap).
Oppure dobbbiamo fare in modo di analizzare per primi gli stati che hanno "pochi" true e molti false, che quindi portano meno overlap. Quindi inserendoli per primi nella priority queue verranno considerati per primi.

In [262]:
assert goal_check((State(set(range(NUM_SETS)), set()))), "Problem not solvable"

In [263]:
def search(distance):
    # We now start the search. This is the starting point.
    frontier = PriorityQueue()  # prio -> dijkstra.
    state = State(set(), set(range(NUM_SETS)))
    frontier.put((distance(state), state))

    counter = 0
    _, current_state = frontier.get()  # take the state
    # check if state is the solution
    while not goal_check(current_state):
        counter += 1
        for action in current_state.not_taken:  # all action we can take now
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )  # | is the set union
            frontier.put((distance(new_state), new_state))
        _, current_state = frontier.get()

    print(
        f"Solved in {counter:,} steps ({len(current_state.taken)} tiles), with state: {current_state.taken}"
    )
    return current_state.taken

In [264]:
sol = search(my_h)

print(overlap(sol))

search(distance_prof)

print(overlap(sol))

Solved in 5 steps (5 tiles), with state: {295, 204, 473, 282, 159}
0.106
Solved in 6 steps (6 tiles), with state: {4, 197, 427, 82, 473, 159}
0.106
